## Przygotowanie Danych (Data Preprocessing)

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
krk = pd.read_csv("..\data_raw\olx_krakow_detailed_listings.csv")
waw = pd.read_csv("..\data_raw\olx_warszawa_detailed_listings.csv")
wro = pd.read_csv("..\data_raw\olx_wroclaw_detailed_listings.csv")
lcj = pd.read_csv("..\data_raw\olx_lodz_detailed_listings.csv")
poz = pd.read_csv("..\data_raw\olx_poznan_detailed_listings.csv")
gdn = pd.read_csv("..\data_raw\olx_gdansk_detailed_listings.csv")
szz = pd.read_csv("..\data_raw\olx_szczecin_detailed_listings.csv")
bzg = pd.read_csv("..\data_raw\olx_bydgoszcz_detailed_listings.csv")
ns = pd.read_csv("..\data_raw\olx_nowy-sacz_detailed_listings.csv")

In [4]:
# połacznie danych
list_city = [krk, waw, wro, lcj, poz, gdn, szz, bzg, ns]

connect_data= pd.DataFrame()

for file in list_city:
    connect_data = pd.concat([connect_data, file], ignore_index= True)

In [5]:
connect_data.sample(10)

,Title,Price,Area,Address,description,table,Link
2238,Wynajmę mieszkanie ul. Kłobucka 6e z komórką i MP,4 500 zł,57 m²,"Warszawa, Ursynów - 04 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/wynajme-mieszk...
7787,Kawalerka z widokiem przy ulicy Ofiar Oświęcimia,1 700 zł,19 m²,"Szczecin, Centrum - Odświeżono Dzisiaj o 11:39",Na wynajem słoneczne kawalerka niedaleko Manha...,PrywatneZwierzęta: TakWinda: TakPoziom: 6Umebl...,https://www.olx.pl/oferta/kawalerka-z-widokiem...
7685,Kawalerka z balkonem i ogródkiem Gdańsk od maja,2 100 zł,32 m²,"Gdańsk, Jasień - 13 marca 2025",Wynajmę kawalerkę w dogodnej lokalizacji. Mies...,PrywatneParking: identyfikator do strefy płatn...,https://www.olx.pl/oferta/kawalerka-z-balkonem...
3412,"Nowe 2-pok. 53m2 wysoki standard, bezpośrednio!",3 300 zł,53 m²,"Wrocław, Śródmieście - 12 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/nowe-2-pok-53m...
7785,POMORZANY. 2 pokoje z osobną kuchnią.,2 400 zł,"52,82 m²","Szczecin, Pomorzany - 24 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/pomorzany-2-po...
727,Mieszkanie dwupokojowe do wynajęcia Kraków Ruczaj,2 500 zł,42 m²,"Kraków, Dębniki - Odświeżono dnia 23 marca 2025",Mam do wynajęcia mieszkanie 2 pokojowe na Rucz...,PrywatneZwierzęta: NieWinda: TakParking: w gar...,https://www.olx.pl/d/oferta/mieszkanie-dwupoko...
2442,Duża kawalerka z sypialnią + balkon Mokotów Sł...,3 100 zł,33 m²,"Warszawa, Mokotów - 23 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/duza-kawalerka...
8563,Ładna kawalerka od 1 Maja w cichym centrum Sz...,1 600 zł,27 m²,"Szczecin, Centrum - 25 lutego 2025",Ładna kawalerka w cichym centrum Szczecina na...,PrywatneZwierzęta: NieWinda: NieParking: przyn...,https://www.olx.pl/d/oferta/ladna-kawalerka-od...
904,2 pokoje | Miesiąc GRATIS | Bez pośredników,2 890 zł,"36,57 m²","Kraków, Podgórze - 11 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/2-pokoje-miesi...
1544,Zamknięte Osiedle| 2 Balkony|Blisko Metra Imielin,4 500 zł,67 m²,"Warszawa, Ursynów - Odświeżono dnia 24 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/zamkniete-osie...


In [6]:
## sprawdzenie danych

In [7]:
connect_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9717 entries, 0 to 9716
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        9717 non-null   object
 1   Price        9717 non-null   object
 2   Area         9717 non-null   object
 3   Address      9717 non-null   object
 4   description  9717 non-null   object
 5   table        9717 non-null   object
 6   Link         9717 non-null   object
dtypes: object(7)
memory usage: 531.5+ KB


In [8]:
connect_data.isnull().sum()

Title          0
Price          0
Area           0
Address        0
description    0
table          0
Link           0
dtype: int64

## Przetwarzanie danych

### Price

In [11]:
type(connect_data['Price'][0])

str

In [12]:
# usuwanie "zł" i tekstu, zamiana na float
connect_data['Price'] = connect_data['Price'].str.replace(r'[^\d]','',regex= True).astype(float)

In [13]:
type(connect_data['Price'][0])

numpy.float64

In [14]:
connect_data['Price'].describe()

count      9717.000000
mean       3973.240403
std       19980.389549
min         180.000000
25%        2000.000000
50%        2500.000000
75%        3000.000000
max      528516.000000
Name: Price, dtype: float64

In [15]:
# usuniecie wartosci odstających
connect_data = connect_data[connect_data['Price'] < 15000]

### Area

In [17]:
# usuwanie "m²" i przecinków, zamiana na float
connect_data['Area'] =connect_data['Area'].str.replace(r',', '.').str.extract(r'([\d\.]+)').astype(float)

In [18]:
connect_data['Area'].describe()

count    9662.000000
mean       42.676558
std        17.880614
min         1.000000
25%        32.000000
50%        40.000000
75%        50.000000
max       440.000000
Name: Area, dtype: float64

In [19]:
 # usuwam wartosc poniewaz jest to biuro 
connect_data = connect_data[connect_data['Area'] < 200]

### Address	

In [21]:
connect_data['city'] = connect_data['Address'].map(lambda x: x.split('-')[0].split(',')[0])
connect_data['city'] = connect_data['city'].map(lambda x: x.strip().lower())

In [22]:
connect_data['city'].value_counts()

city
warszawa                 1293
kraków                   1292
wrocław                  1288
łódź                     1286
poznań                   1280
gdańsk                   1278
szczecin                 1028
bydgoszcz                 618
tarnów                     63
nowy sącz                  51
bochnia                    26
niepołomice                24
brzesko                    23
gorlice                    21
toruń                      19
mszana dolna               11
gdów                        8
sępólno krajeńskie          4
wiśniowa                    4
wieliczka                   3
czarnochowice               3
janowiec wielkopolski       2
podłęże                     2
dobczyce                    2
stróżówka                   1
papowo toruńskie            1
ochmanów                    1
grębocin                    1
trąbki                      1
bukowina tatrzańska         1
łysokanie                   1
białka tatrzańska           1
kłaj                        1
świec

In [23]:
connect_data['district'] = connect_data['Address'].map(lambda x: x.split('-')[0])
#connect_data['district'] = connect_data['district'].map(lambda x: x.strip().lower())

In [24]:
def point(x):
    if ',' in x:
        return x.split(',')[1].strip().lower()
    else:
        return x.lower()

In [25]:
connect_data['district'] = connect_data['district'].map(lambda x: point(x))

In [26]:
connect_data[['Address','district','city']].sample(10)

,Address,district,city
7868,"Szczecin, Centrum - 17 marca 2025",centrum,szczecin
4221,"Łódź, Bałuty - 21 marca 2025",bałuty,łódź
1230,"Kraków, Dębniki - 14 marca 2025",dębniki,kraków
213,"Kraków, Krowodrza - Odświeżono dnia 17 marca 2025",krowodrza,kraków
1487,"Warszawa, Białołęka - Odświeżono dnia 24 marca...",białołęka,warszawa
5843,"Poznań, Stare Miasto - 24 marca 2025",stare miasto,poznań
7390,"Gdańsk, Oliwa - 27 lutego 2025",oliwa,gdańsk
7884,Szczecin - 17 marca 2025,szczecin,szczecin
7882,"Szczecin, Świerczewo - 23 marca 2025",świerczewo,szczecin
8929,Bydgoszcz - 22 marca 2025,bydgoszcz,bydgoszcz


In [27]:
#krk['time_part'] = krk['Address'].map(lambda x: x.split('-')[1])

### table

In [29]:
# usuniecie rekordów ktre nie maja opisów
connect_data = connect_data[connect_data['table'] !='brak opisu otodom']

In [30]:
connect_data = connect_data[connect_data['table'] != 'brak danych']

In [31]:
connect_data.shape

(5162, 9)

In [32]:
connect_data['table_new'] = connect_data['table'].astype(str).str.findall(r'[A-Z][^A-Z]*').str.join(' ')


In [33]:
connect_data['table_new'] = connect_data['table_new'].str.replace(r'\s{2,}', ' ', regex=True)

In [34]:
connect_data['table_new'][1]

'Prywatne Zwierzęta: Nie Winda: Tak Parking: przynależne na ulicy Poziom: 1 Umeblowane: Tak Rodzaj zabudowy: Apartamentowiec Powierzchnia: 38 m² Liczba pokoi: 2 pokoje Czynsz (dodatkowo): 370 zł'

In [35]:
connect_data['Prywatne'] = connect_data['table_new'].str.contains('Prywatne').astype(int)
connect_data['Firmowe'] = connect_data['table_new'].str.contains('Firmowe').astype(int)
connect_data['Zwierzęta'] = connect_data['table_new'].str.contains('Zwierzęta: Tak').astype(int)
connect_data['Winda'] = connect_data['table_new'].str.contains('Winda: Tak').astype(int)
connect_data['Umeblowane'] = connect_data['table_new'].str.contains('Umeblowane: Tak').astype(int)

In [77]:
def level(x):
    x= x.lower()
    before, sep, after = x.partition('poziom:')
    if not sep:
        return 'brak'
    level = after.strip().lower() if sep else x.lower()
    before, sep, after =level.partition('umeblowane:')
    return before
connect_data['Poziom']= connect_data['table_new'].map(lambda x: level(x))

In [93]:
def level(x):
    x= x.lower()
    before, sep, after = x.partition('poziom:')
    if not sep:
        return after
    level = after.strip().lower() if sep else x.lower()
    before, sep, after =level.partition('umeblowane:')
    return before
connect_data['Poziom']= connect_data['table_new'].map(lambda x: level(x))

In [95]:
connect_data['Poziom'].value_counts()

Poziom
1              1132
2               968
parter          785
3               769
4               572
5               214
                204
6               121
7               102
10               82
9                81
8                56
powyżej 10       55
suterena         16
poddasze          5
Name: count, dtype: int64

In [38]:
def rooms(x):
    x= x.lower()
    before, sep, after = x.partition('liczba pokoi:')
    if not sep:
        return -1
    rooms = after.strip().lower() if sep else x.lower()
    before, sep, after =rooms.partition('czynsz')
    return before.strip()
connect_data['Pokoje']= connect_data['table_new'].map(lambda x: rooms(x))

In [39]:
connect_data[connect_data['Pokoje'] ==-1]

,Title,Price,Area,Address,description,table,Link,city,district,table_new,Prywatne,Firmowe,Zwierzęta,Winda,Umeblowane,Poziom,Pokoje
924,dwupoziomowe ul. zachodnia kraków ruczaj 77m2,3600.0,77.0,"Kraków, Dębniki - 07 marca 2025","NOWOCZESNY, SŁONECZNY, PRAKTYCZNIE URZĄDZONY, ...",PrywatnePowierzchnia: 77 m²,https://www.olx.pl/d/oferta/dwupoziomowe-ul-za...,kraków,dębniki,Prywatne Powierzchnia: 77 m²,1,0,0,0,0,-1,-1


In [40]:
connect_data = connect_data[connect_data['Pokoje'] != -1]

In [41]:
Rooms = {'2 pokoje': 2, 'kawalerka': 1, '3 pokoje': 3, '4 i więcej':4}
connect_data['Rooms'] = connect_data['Pokoje'].map(Rooms)

In [42]:
connect_data['Rooms'].value_counts()

Rooms
2    2513
1    1790
3     747
4     111
Name: count, dtype: int64

In [43]:
def rent_extra(x):
    x= x.lower()
    before, sep, after = x.partition('czynsz (dodatkowo): ')
    if not sep:
        return 0
    return after
connect_data['czynsz']= connect_data['table_new'].map(lambda x: rent_extra(x))
connect_data['czynsz'] = connect_data['czynsz'].replace(r'[^\d]','',regex= True).astype(float)

In [44]:
connect_data['czynsz'].value_counts()

czynsz
0.0       829
600.0     404
500.0     356
400.0     222
700.0     212
         ... 
571.0       1
714.0       1
1010.0      1
852.0       1
165.0       1
Name: count, Length: 366, dtype: int64

In [45]:
def parking(x):
    x= x.lower()
    before, sep, after = x.partition('parking:')
    if not sep:
        return 'brak'
    spot = after.strip().lower() if sep else x.lower()
    before, sep, after =spot.partition('poziom:')
    if not sep:
        before, sep, after =spot.partition('umeblowane:')
        return before
    return before.strip()
connect_data['Parking']= connect_data['table_new'].map(lambda x: parking(x))

In [46]:
connect_data['Parking'] = connect_data['Parking'].apply(
    lambda x: 'garaż' if isinstance(x, str) and ('garażu' in x.lower() or 'garazu' in x.lower()) else x
)

In [47]:
connect_data['Parking'] = connect_data['Parking'].apply(
    lambda x: 'strzeżony' if isinstance(x, str) and ('strzeżony' in x.lower() or 'strzezony' in x.lower()) else x
)

In [48]:
connect_data['Parking'] = connect_data['Parking'].apply(
    lambda x: 'ulica' if isinstance(x, str) and ('ulicy' in x.lower()) else x
)

In [49]:
connect_data['Parking'] = connect_data['Parking'].apply(
    lambda x: 'brak' if isinstance(x, str) and ('brak' in x.lower() or '0' in x.lower()) else x
)

In [50]:
connect_data['Parking'].value_counts()

Parking
brak                                            2416
ulica                                           1269
garaż                                           1051
strzeżony                                        301
identyfikator do strefy płatnego parkowania      117
identyfikator do strefy płatnego parkowania        7
Name: count, dtype: int64

In [51]:
def development (x):
    x= x.lower()
    before, sep, after = x.partition('rodzaj zabudowy:')
    if not sep:
        return -1
    built = after.strip().lower() if sep else x.lower()
    before, sep, after =built.partition('powierzchnia:')
    return before
connect_data['Zabudowa']= connect_data['table_new'].map(lambda x: development(x))

In [52]:
connect_data['Zabudowa'].value_counts()

Zabudowa
blok                 2750
apartamentowiec      1166
kamienica             888
dom wolnostojący      203
szeregowiec            66
pozostałe              66
loft                   22
Name: count, dtype: int64

In [53]:
connect_data['Total_price'] = connect_data['Price'] + connect_data['czynsz']

In [54]:
connect_data[['Total_price','Price','czynsz']]

,Total_price,Price,czynsz
0,3900.0,3250.0,650.0
1,3670.0,3300.0,370.0
2,3300.0,2800.0,500.0
3,2100.0,2100.0,0.0
4,2600.0,2300.0,300.0
...,...,...,...
9711,1600.0,1200.0,400.0
9713,2330.0,2000.0,330.0
9714,2900.0,2500.0,400.0
9715,3550.0,3100.0,450.0


### description    

In [56]:
krk['description'][0]

'Do wynajęcia trzypokojowe mieszkanie o powierzchni 68,4 m2 przy ul. Bajecznej 2b.\nPosiada dwa balkony, z wyjściem zarówno z sypialni jak i salonu.\nMieszkanie po odświeżeniu, posiada podstawowe wyposażenie. \n\nLOKALIZACJA:\nSklep spożywczy “Biedronka” - 100m\nGaleria M1 - 15 min komunikacją miejską\nGaleria Kazimierz - 20 min komunikacją miejską\nRynek – 20 min komunikacją miejską\nDworzec główny - 20 min komunikacją miejską\n\nKOMUNIKACJA:\nPrzystanek tramwajowy: Ofiar Dąbia w odległości ok. 300m.\nPrzystanek autobusowy: Ofiar Dąbia w odległości ok. 250m.\n\nMieszkanie składa się z:\n\n- dużego salonu z balkonem,\n- gabinetu z szafą, biurkiem i krzesłem biurowym,\n- dużej sypialni z wyjściem na balkon,\n- osobnej kuchni z podstawowym wyposażeniem (Kuchenka mikrofalowa, płyta elektryczna, lodówka),\n- Łazienki z pralką, umywalką, oraz kabiną prysznicową, \n- osobnej toalety wraz z umywalką,\n- Przestronnego korytarza.\n\nCENA:\nMiesięczny czynsz najmu: 3250 zł\nCzynsz administracyjn

In [57]:
# Custom function to clean finance-related text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '[NUM]', text)  # Replace numbers with [NUM]
    text = re.sub(r'\$+', '[CURRENCY]', text)  # Replace currency symbols with a placeholder
    return text

In [58]:
krk['Clean_Description'] = krk['description'].apply(clean_text)

In [59]:
krk['Clean_Description'][0]

'do wynajęcia trzypokojowe mieszkanie o powierzchni [NUM],[NUM] m[NUM] przy ul. bajecznej [NUM]b.\nposiada dwa balkony, z wyjściem zarówno z sypialni jak i salonu.\nmieszkanie po odświeżeniu, posiada podstawowe wyposażenie. \n\nlokalizacja:\nsklep spożywczy “biedronka” - [NUM]m\ngaleria m[NUM] - [NUM] min komunikacją miejską\ngaleria kazimierz - [NUM] min komunikacją miejską\nrynek – [NUM] min komunikacją miejską\ndworzec główny - [NUM] min komunikacją miejską\n\nkomunikacja:\nprzystanek tramwajowy: ofiar dąbia w odległości ok. [NUM]m.\nprzystanek autobusowy: ofiar dąbia w odległości ok. [NUM]m.\n\nmieszkanie składa się z:\n\n- dużego salonu z balkonem,\n- gabinetu z szafą, biurkiem i krzesłem biurowym,\n- dużej sypialni z wyjściem na balkon,\n- osobnej kuchni z podstawowym wyposażeniem (kuchenka mikrofalowa, płyta elektryczna, lodówka),\n- łazienki z pralką, umywalką, oraz kabiną prysznicową, \n- osobnej toalety wraz z umywalką,\n- przestronnego korytarza.\n\ncena:\nmiesięczny czynsz 

### title

In [61]:
krk['Title_clear'] = krk['Title'].apply(clean_text)

In [62]:
krk['Title_clear']

0       mieszkanie [NUM]-pokojowe - ul. bajeczna [NUM]...
1       nowe/klima/ [NUM]m[NUM]/[NUM] pokoje/ wysoki s...
2       mieszkanie z klimatyzacją od zaraz | dąbska | ...
3       nowoczesna kawalerka ul. monte cassino /[NUM]m...
4                   studio pierwszy najem [NUM] zł ruczaj
                              ...                        
1291    [NUM] pokoje - parking - osobna kuchnia - wietora
1292    [NUM] pokoje! - nowa [NUM] dzielnica ul. wrocł...
1293    kawalerka [NUM] m[NUM] czyżyny sołtysowska osi...
1294            dwupokojowe mieszkanie, dostępne od zaraz
1295    [NUM]-pokojowe mieszkanie, [NUM]m[NUM], ul. ba...
Name: Title_clear, Length: 1296, dtype: object

### usuniecie kolumny zbednych kolumn

In [64]:
connect_data.head(5)

,Title,Price,Area,Address,description,table,Link,city,district,table_new,...,Zwierzęta,Winda,Umeblowane,Poziom,Pokoje,Rooms,czynsz,Parking,Zabudowa,Total_price
0,Mieszkanie 3-pokojowe - ul. Bajeczna 2B - Grze...,3250.0,68.4,"Kraków, Grzegórzki - Odświeżono dnia 24 marca ...",Do wynajęcia trzypokojowe mieszkanie o powierz...,FirmoweZwierzęta: NieWinda: TakPoziom: 4Umeblo...,https://www.olx.pl/oferta/mieszkanie-3-pokojow...,kraków,grzegórzki,Firmowe Zwierzęta: Nie Winda: Tak Poziom: 4 Um...,...,0,1,1,4,3 pokoje,3,650.0,brak,blok,3900.0
1,NOWE/Klima/ 38m2/2 pokoje/ wysoki standard/mie...,3300.0,38.0,"Kraków, Czyżyny - 23 marca 2025",MIESZKANIE:\nDo wynajęcia nowocześnie urządzon...,PrywatneZwierzęta: NieWinda: TakParking: przyn...,https://www.olx.pl/oferta/nowe-klima-38m2-2-po...,kraków,czyżyny,Prywatne Zwierzęta: Nie Winda: Tak Parking: pr...,...,0,1,1,1,2 pokoje,2,370.0,ulica,apartamentowiec,3670.0
2,Mieszkanie z klimatyzacją od zaraz | Dąbska | ...,2800.0,40.0,"Kraków, Grzegórzki - 24 marca 2025",ENG BELOW\nPromo Estates przedstawia ofertę wy...,FirmoweZwierzęta: TakWinda: TakParking: w gara...,https://www.olx.pl/oferta/mieszkanie-z-klimaty...,kraków,grzegórzki,Firmowe Zwierzęta: Tak Winda: Tak Parking: w g...,...,1,1,1,2,2 pokoje,2,500.0,garaż,blok,3300.0
3,Nowoczesna kawalerka ul. Monte Cassino /100m o...,2100.0,20.0,"Kraków, Stare Miasto - Odświeżono Dzisiaj o 10:10",English down below.\n\nPrezentuję na wynajem n...,PrywatneZwierzęta: NieWinda: NiePoziom: 1Umebl...,https://www.olx.pl/oferta/nowoczesna-kawalerka...,kraków,stare miasto,Prywatne Zwierzęta: Nie Winda: Nie Poziom: 1 U...,...,0,0,1,1,kawalerka,1,0.0,brak,kamienica,2100.0
4,Studio pierwszy najem 2300 zł Ruczaj,2300.0,22.0,"Kraków, Dębniki - Odświeżono Dzisiaj o 10:19",Cena 2600 zł z czynszem + indywidualne opłaty ...,PrywatneZwierzęta: TakWinda: TakParking: przyn...,https://www.olx.pl/oferta/studio-pierwszy-naje...,kraków,dębniki,Prywatne Zwierzęta: Tak Winda: Tak Parking: pr...,...,1,1,1,4,kawalerka,1,300.0,ulica,blok,2600.0


In [65]:
# usuniecie kolumny 
connect_data.drop(columns=['Address','Link', 'Title','description','table','table_new','Price','czynsz','Pokoje'], axis=1, inplace= True)

In [66]:
connect_data

,Area,city,district,Prywatne,Firmowe,Zwierzęta,Winda,Umeblowane,Poziom,Rooms,Parking,Zabudowa,Total_price
0,68.4,kraków,grzegórzki,0,1,0,1,1,4,3,brak,blok,3900.0
1,38.0,kraków,czyżyny,1,0,0,1,1,1,2,ulica,apartamentowiec,3670.0
2,40.0,kraków,grzegórzki,0,1,1,1,1,2,2,garaż,blok,3300.0
3,20.0,kraków,stare miasto,1,0,0,0,1,1,1,brak,kamienica,2100.0
4,22.0,kraków,dębniki,1,0,1,1,1,4,1,ulica,blok,2600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9711,20.0,wieliczka,wieliczka,1,0,0,0,1,1,1,strzeżony,dom wolnostojący,1600.0
9713,25.0,czarnochowice,czarnochowice,1,0,0,1,1,1,1,garaż,apartamentowiec,2330.0
9714,27.0,czarnochowice,czarnochowice,1,0,1,1,1,4,1,garaż,blok,2900.0
9715,61.0,czarnochowice,czarnochowice,0,1,0,1,1,1,4,garaż,blok,3550.0


In [125]:
connect_data.to_csv('..\data\connect_data.csv')